# Banane Group Creator 

### Packages

In [1]:
import json
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
from networkx.algorithms.bipartite import minimum_weight_full_matching
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_bipartite_matching

### Constants

In [101]:
DAYS = {'MONDAY':0,'TUESDAY':1,'WEDNESDAY':2,'THURSDAY':3,'FRIDAY':4}
NAME_FIELD = 'Prenom-Nom'
GROUP_MEMBER_FIELD = 'Membre_1'
EXEMPLE_NAME = 'Jean-Mich'
VALID_TOKEN = 'oui'
GROUP_SIZE = 5
TEMPERATURE = 1

### Data

In [66]:
preferences = {'Maria':{'MONDAY','TUESDAY'},'Boris':{'WEDNESDAY'},
              'Jeremy':{'TUESDAY'},'Farah':{'WEDNESDAY','THURSDAY'},
              'Gabriel':{'WEDNESDAY','TUESDAY'},'Alexia':{'THURSDAY'},
              'Ines':{'THURSDAY','TUESDAY'},'Shayan':{'WEDNESDAY','FRIDAY'},
              'Noah':{'MONDAY','TUESDAY','FRIDAY'},'Mariella':{'WEDNESDAY','MONDAY'},
              'Antoine':{'MONDAY','WEDNESDAY'},'Etienne':{'WEDNESDAY'},
              'Valentine':{'THURSDAY','TUESDAY'},'Bastien':{'WEDNESDAY','MONDAY','THURSDAY','FRIDAY'},
              'Benoit':{'MONDAY','TUESDAY'},'Billibob':{'MONDAY'},
              'Juliette':{'MONDAY','FRIDAY'},'Flavia':{'WEDNESDAY','MONDAY'},
              'Amelie':{'TUESDAY','FRIDAY','WEDNESDAY'},'Gaetan':{'WEDNESDAY','MONDAY','TUESDAY'},}

In [77]:
prior_groups = [{'Juliette','Benoit','Antoine'},]

### Helpers

In [162]:
def parse_preference_file(input_path:str) -> dict:
    """
    Convert preferences from csv format to dictionnary.
    
    :param input_path: Path to the preferences in csv format.
    
    :return: Dictionnary with names as keys and days as values.
    
    """
    pref_df = pd.read_csv(input_path)
    preferences = dict()
    for _, row_data in pref_df.iterrows():
        if row_data[NAME_FIELD] != EXEMPLE_NAME:
            pref_days = [d for d in DAYS.keys() if row_data[d] == VALID_TOKEN]
            preferences[row_data[NAME_FIELD]] = set(pref_days)
    return preferences

def parse_group_file(input_path:str) -> list:
    """
    Convert group choices from csv format to list of sets.
    
    :param input_path: Path to the preferences in csv format.
    
    :return: List of groups represented as sets.
    
    """
    group_df = pd.read_csv(input_path)
    prior_groups = []
    for _, row_data in group_df.iterrows():
        if row_data[GROUP_MEMBER_FIELD] != EXEMPLE_NAME:
            prior_groups.extend(set(row_data.tolist()))

def create_tokens(preferences:dict, prior_groups:list)-> dict:
    """
    Create people token to merge people from groups.
    
    Tokens are used to facilitate matching. If you are not
    coming from a prior group then your token is your name.
    In the other case, we create $N$ similar tokens that 
    mimic the group members, where $N$ is the number of 
    people in the prior group.
    
    :param preferences: Dictionnary with people as keys and days as values.
    :param prior_groups: List of groups (represented by a set).
    
    :return: Dictionnary in same format as preferences but for tokens.
    
    """
    # Init token preferences
    token_preferences = dict()
    # Create group token and preferences
    members_of_prior_groups = []
    for g in prior_groups:
        if len(g) < 1:
            raise ValueError('Empty group provided in the prior data.')
        group = list(g)
        overlap_days = set().union(preferences[group[0]])
        for name in group:
            overlap_days = overlap_days.intersection(preferences[name])
        if len(overlap_days) == 0:
            raise Exception(f'Impossible to match friends preferences for group {g} [no overlapping days].')
        members_of_prior_groups.extend(group)
        group_token = '/'.join(group)
        for i in range(len(group)):
            token_preferences[group_token+f'/{i}'] = overlap_days
    members_of_prior_groups = set(members_of_prior_groups)
    # Create other people token
    for name, pref in preferences.items():
        if name not in members_of_prior_groups:
            token_preferences[name] = pref
    return token_preferences

def create_mappings(token_preferences:dict)->tuple:
    """
    Create people and day mappings.
    
    :param token_preferences: Dictionnary with token as keys and days as values.
    
    :return: Mappings and reversed mappings for token-id and day-id.
    
    """
    # Standard Mappings
    token_mapping = dict(zip(token_preferences.keys(), range(len(token_preferences))))
    day_mapping = [d+f'_{i+1}' for d in DAYS.keys() for i in range(GROUP_SIZE)]
    day_mapping = dict(zip(day_mapping,range(len(day_mapping))))
    # Reversed Mappings
    reversed_token_mapping = dict(zip(token_mapping.values(),token_mapping.keys()))
    reversed_day_mapping = dict(zip(day_mapping.values(),day_mapping.keys()))
    return token_mapping, reversed_token_mapping, day_mapping, reversed_day_mapping

def create_token_edges(token_preferences:dict)-> dict:
    """
    Create weighted edges between tokens and day's slots.
    
    :param token_preferences: Dictionnary with token as keys and days as values.
    
    :return: Dictionnary with (token,day) pairs as keys and weights as values.
    
    """
    weights = dict()
    for token, days in token_preferences.items():
        for d in days:
            for i in range(GROUP_SIZE):
                weights[(token,d+f'_{i+1}')] = 1/len(days) + (len(token.split('_'))-1)*TEMPERATURE
    return weights

def create_graph_and_adj_matrix(weights:dict, token_mapping:dict,
                                day_mapping:dict)->tuple:
    """
    Create the networkx graph and adjacency matrix from the given edge weights.
    
    :param weights: Dictionnary with (token,day) pairs as keys and weights as values.
    :param token_mapping: Dictionnary with tokens as keys and ids as values.
    :param day_mapping: Dictionnary with days as keys and ids as values.
    
    :return: Tuple in form (Networkx Graph, adjacency matrix)
    
    """
    # Convert to pandas DataFrame
    weighted_preferences = [[*a,b] for a,b in weights.items()]
    weighted_preferences_df = pd.DataFrame(weighted_preferences,columns=['name','day','weight'])
    # Populate Adjacency Matrix
    adjacency_matrix = np.zeros((len(token_mapping),len(day_mapping)),dtype=np.float32)
    adjacency_matrix[weighted_preferences_df['name'].apply(lambda p: token_mapping[p]).to_list(),
                 weighted_preferences_df['day'].apply(lambda p: day_mapping[p]).to_list(
                 )] = weighted_preferences_df['weight'].to_list()
    # Convert to sparse format
    adjacency_matrix = csr_matrix(adjacency_matrix)
    # Create Graph
    G = nx.from_pandas_edgelist(weighted_preferences_df,source='name',target='day',
                            edge_attr='weight',create_using=nx.MultiGraph)
    return G, adjacency_matrix

def format_matching(matching:list)->pd.DataFrame:
    """
    Format the matching in (day, name) tuples and store in pandas DataFrame.
    
    :param matching: List of (token,day) pairs resulting from matching.
    
    :return: Pandas DataFrame with the matching data. 
    
    """
    final_matching = dict()
    # Format Matching
    for a,b in matching:
        # Format Day
        if '_' in a:
            day, token_name = a.split('_')[0], b
        else:
            day, token_name = b.split('_')[0], a
        # Format token name
        if '/' in token_name:
            name = token_name.split('/')[int(token_name.split('/')[-1])]
        else:
            name = token_name
        # Add entry
        if day not in final_matching:
            final_matching[day] = []
        final_matching[day].append(name)
    # Sort and fill up entries to have consistent size in DataFrame
    for day in final_matching.keys():
        final_matching[day] = sorted(final_matching[day])
        people = final_matching[day]
        if len(people) < GROUP_SIZE:
            people.extend(['']*(GROUP_SIZE-len(people)))
    # Convert to pandas DataFrame
    final_matching_df = pd.DataFrame(final_matching).T
    final_matching_df = final_matching_df.sort_index(key=lambda idx: [DAYS[i] for i in idx])
    return final_matching_df

def pipeline(preferences:dict,prior_groups:list,matching_type='max_weight')-> pd.DataFrame:
    """
    Pipeline for group creation.
    
    :param preferences: Dictionnary with people as keys and days as values.
    :param prior_groups: List of groups (represented by a set).
    :param matching_type: Matching algorithm, to be selected among the following list
                          ['max_weight','bipartite','scipy']. The best and default is 
                          'max weight'.
    
    :return: Pandas DataFrame with the matching data. 
    
    """
    token_preferences = create_tokens(preferences,prior_groups)
    (token_mapping, reversed_token_mapping, day_mapping,
     reversed_day_mapping) = create_mappings(token_preferences)
    weights = create_token_edges(token_preferences)
    G, adj = create_graph_and_adj_matrix(weights,
                                token_mapping,day_mapping)
    if matching_type == 'bipartite':
        for i,j,k in list(G.edges):
            G[i][j][k]['weight'] = -G[i][j][k]['weight']
        matching = minimum_weight_full_matching(G,weight='weight')
        matching = list(zip(matching.keys(),matching.values()))
        matching = list(set([(a,b) if '_' in a else (b,a) for (a,b) in matching]))
    elif matching_type == 'max_weight':
        matching = list(nx.max_weight_matching(G,maxcardinality=True))
    elif matching_type == 'scipy':
        matching = maximum_bipartite_matching(adj, perm_type='column')
        matching = [(reversed_token_mapping[i], reversed_day_mapping[matching[i]]) 
                    for i in range(len(matching))]
    else:
        raise ValueError(f'Matching algorithm {matching_type} not implemented')
    return format_matching(matching)

### Results

In [163]:
pipeline(preferences,prior_groups,matching_type='max_weight')

,0,1,2,3,4
MONDAY,Antoine,Benoit,Billibob,Juliette,
TUESDAY,Gabriel,Gaetan,Jeremy,Maria,
WEDNESDAY,Boris,Etienne,Flavia,Mariella,
THURSDAY,Alexia,Farah,Ines,Valentine,
FRIDAY,Amelie,Bastien,Noah,Shayan,


In [164]:
pipeline(preferences,prior_groups,matching_type='bipartite')

,0,1,2,3,4
MONDAY,Antoine,Bastien,Benoit,Billibob,Juliette
TUESDAY,Gabriel,Gaetan,Jeremy,Maria,Noah
WEDNESDAY,Boris,Etienne,Flavia,Mariella,
THURSDAY,Alexia,Farah,Ines,Valentine,
FRIDAY,Amelie,Shayan,,,


In [165]:
pipeline(preferences,prior_groups,matching_type='scipy')

,0,1,2,3,4
MONDAY,Antoine,Benoit,Billibob,Juliette,Maria
TUESDAY,Amelie,Gabriel,Ines,Jeremy,Valentine
WEDNESDAY,Boris,Etienne,Flavia,Gaetan,Mariella
THURSDAY,Alexia,Bastien,Farah,,
FRIDAY,Noah,Shayan,,,


---

### Archive

In [117]:
available_slots = []
for name, days in preferences.items():
    for i in range(GROUP_SIZE):
        available_slots.extend(zip([name]*len(days),
                                   [d+f'_{i+1}' for d in list(days)],
                                    [1/len(days)]*len(days)))
for name, friends in prior.items():
    overlap_days = set().union(preferences[name])
    for f in friends:
        overlap_days = overlap_days.intersection(preferences[f])
    if len(overlap_days) == 0:
        raise Exception(f'Impossible to match friends preferences for {name} as no overlapping days.')
    for d in list(overlap_days):
        for i in range(GROUP_SIZE):
            available_slots.extend(zip([name]*len(overlap_days),
                                       [d+f'_{i+1}' for d in list(overlap_days)],
                                        [TEMPERATURE/len(overlap_days)]*len(overlap_days)))

Data Format:
- adjacency matrix NxM. 
- N is the number of people
- M is the number of slots (#days*group_size) (ordered)

In [5]:
# Standard Mappings
people_mapping = dict(zip(preferences.keys(), range(len(preferences))))
day_mapping = [d+f'_{i+1}' for d in DAYS.keys() for i in range(GROUP_SIZE)]
day_mapping = dict(zip(day_mapping,range(len(day_mapping))))
# Reversed Mappings
reversed_people_mapping = dict(zip(people_mapping.values(),people_mapping.keys()))
reversed_day_mapping = dict(zip(day_mapping.values(),day_mapping.keys()))
adjacency_matrix = np.zeros((len(people_mapping),len(day_mapping)),dtype=np.float32)

#### Create weighted edges

In [150]:
weights = dict()
for name, days in preferences.items():
    for d in days:
        for i in range(GROUP_SIZE):
            weights[(name,d+f'_{i+1}')] = 1/len(days) 

In [151]:
for name, friends in prior.items():
    overlap_days = set().union(preferences[name])
    for f in friends:
        overlap_days = overlap_days.intersection(preferences[f])
    if len(overlap_days) == 0:
        raise Exception(f'Impossible to match friends preferences for {name} as no overlapping days.')
    for d in list(overlap_days):
        for i in range(GROUP_SIZE):
            weights[(name,d+f'_{i+1}')] = weights[(name,d+f'_{i+1}')] + TEMPERATURE/len(overlap_days)

AttributeError: 'list' object has no attribute 'items'

#### Convert to pandas DataFrame

In [93]:
weighted_preferences = [[*a,b] for a,b in weights.items()]

In [94]:
weighted_preferences_df = pd.DataFrame(weighted_preferences,columns=['name','day','weight'])
weighted_preferences_df.head()

,name,day,weight
0,Maria,MONDAY_1,0.5
1,Maria,MONDAY_2,0.5
2,Maria,MONDAY_3,0.5
3,Maria,MONDAY_4,0.5
4,Maria,MONDAY_5,0.5


#### Populate adjacency matrix

In [95]:
adjacency_matrix[weighted_preferences_df['name'].apply(lambda p: people_mapping[p]).to_list(),
                 weighted_preferences_df['day'].apply(lambda p: day_mapping[p]).to_list(
                 )] = weighted_preferences_df['weight'].to_list()

In [96]:
adjacency_matrix = csr_matrix(adjacency_matrix)

#### Scipy Matching

In [97]:
matching = maximum_bipartite_matching(adjacency_matrix, perm_type='column')
matching = [(reversed_people_mapping[i], reversed_day_mapping[matching[i]]) for i in range(len(matching))]

In [98]:
matching = maximum_bipartite_matching(adjacency_matrix, perm_type='row')
matching = [(reversed_people_mapping[matching[i]], reversed_day_mapping[i]) for i in range(len(matching))
           if matching[i] != -1]

#### Networkx Matching

In [99]:
G = nx.from_pandas_edgelist(weighted_preferences_df,source='name',target='day',
                            edge_attr='weight',create_using=nx.MultiGraph)
matching = list(nx.max_weight_matching(G,maxcardinality=True))

In [100]:
matching = minimum_weight_full_matching(G,weight='weight')
matching = list(zip(matching.keys(),matching.values()))
matching = list(set([(a,b) if '_' in a else (b,a) for (a,b) in matching]))

#### Format Matching

In [101]:
final_matching = dict()
for a,b in matching:
    if '_' in a:
        day, name = a.split('_')[0], b
    else:
        day, name = b.split('_')[0], a
    if day not in final_matching:
        final_matching[day] = []
    final_matching[day].append(name)
for day in final_matching.keys():
    final_matching[day] = sorted(final_matching[day])
    people = final_matching[day]
    if len(people) < GROUP_SIZE:
        people.extend(['']*(GROUP_SIZE-len(people)))

In [102]:
final_matching_df = pd.DataFrame(final_matching).T
final_matching_df = final_matching_df.sort_index(key=lambda idx: [DAYS[i] for i in idx])